In [25]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
############################
#File Name:
#Author: yanbin
#Mail: yanbin918@gmail.com
#Created Time: 2017-08-10 10:33:09
############################
import math ,operator,random
import sys
sys.path.append('../')
from ml_latest_small.data import *

class ItemCFModel(object):   
    def __init__(self,train,W,K=5,N=5):
        self.train = train
        self.W = W
        self.K = K
        self.N = N

    def Recommendation(self,user_id,config=None):
        W = self.W
        K = self.K
        N = self.N
        if config is not None:
            try:
                K = config['K']
                N = config['N']
            except:
                print "config = {'K':nearest nums,'N':topN nums}"
                break
        train = self.train
        rank=dict()
        if user_id not in train:
            print 'The user is not in traindata'
            return
        ru = train[user_id]
        for i,rui in ru.items():
            for j,wij in sorted(W[i].items(),key=operator.itemgetter(1),reverse=True)[0:K]:
                if j in ru:
                    continue
                if j not in rank:
                    rank[j]=0
                rank[j]+=wij*rui
        topN = sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]
        return topN





class ItemCF(object):
    def ItemSimilarity(self,train):            
#calculate co-rated items between users
        C = dict()
        N = dict()
        for user,items in train.items():
            for i in items:
                if i not in C:
                    C[i]=dict()
                if i not in N:
                    N[i]=0
                N[i] +=1
                for j in items:
                    if i==j:
                        continue
                    if j not in C[i]:
                        C[i][j]=0
                    C[i][j]+=1
        #calculate finial similarity matrix W
        W = dict()
        for i,related_items in C.items():
            W[i]=dict()
            for j,cij in related_items.items():
                W[i][j] = cij/math.sqrt(N[i] * N[j])
        return W

    def ItemSimilarity2(self,train):
        #calculate co-rated items between users
        C = dict()
        N = dict()
        for user,items in train.items():
            for i in items:
                if i not in C:
                    C[i]=dict()
                if i not in N:
                    N[i]=0
                N[i] +=1
                for j in items:
                    if i==j:
                        continue
                    if j not in C[i]:
                        C[i][j]=0
                    C[i][j]+=1/math.log(1+len(items)*1.0)
        #calculate finial similarity matrix W
        W = dict()
        for i,related_items in C.items():
            W[i]=dict()
            for j,cij in related_items.items():
                W[i][j] = cij/math.sqrt(N[i] * N[j])
        return W
    
   
    def train(self,train,Normalization=False):
        W = self.ItemSimilarity2(train)
        if Normalization :
            W = self.Normalization(W)
        return ItemCFModel(train,W)
        
        
    def Normalization(self,W):
        for i,items in W.items():
            m = max(items.values())
            for j,wij in items.items():
                W[i][j]=wij/m
        return W

if __name__ == '__main__':
    itemCF = ItemCF()
    inputPath = "../ml_latest_small/ratings.csv"
    #data = itemCF.loadData(inputPath)
    train_dataset,test_dataset = read_data_sets(inputPath,with_split=True)
    # data = {'A':{'a':1,'b':1,'d':1},'B':{'b':1,'c':1,'e':1},'C':{'d':1,'d':1},'D':{'b':1,'d':1,'d':1},'E':{'a':1,'d':1}}
    train = train_dataset.user_item_dict
    ItemCF_model=itemCF.train(train)
    ItemCF_model.Recommendation(user_id='405')

[('2571', 1.245807792522649),
 ('6539', 0.7473777034628689),
 ('165', 0.7073195868478811),
 ('260', 0.5748515440214882),
 ('592', 0.4578219282347661)]

In [ ]:
class ItemCFModel(object):   
    def __init__(self,train,W,K=5,N=5):
        self.train = train
        self.W = W
        self.K = K
        self.N = N

    def Recommendation(self,user_id):
        W = self.W
        train = self.train
        rank=dict()
        if user_id not in train:
            print 'The user is not in traindata'
        ru = train[user_id]
        for i,rui in ru.items():
            for j,wij in sorted(W[i].items(),key=operator.itemgetter(1),reverse=True)[0:K]:
                if j in ru:
                    continue
                if j not in rank:
                    rank[j]=0
                rank[j]+=wij*rui
        topN = sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]
        return topN




In [ ]:
import sys 
sys.path.append('/home/yanbin/pythonProject/DeepLearning/recommendation/')
from MoiveDisplay.MovieDisplayer import *
movieDisplayer = MovieDisplayer()
movieDisplayer.displayMoive(int(imbd['405']))
movieDisplayer.displayMoive(int(imbd[topN[0][0]]))